In [ ]:
from pathlib import Path

In [ ]:
class logger():
    def __init__(self, prefix=None, *, print=print, file=None, display=True):
        self.print = print
        
        if not isinstance(print, __class__): # print is the raw print
            self._queue = []
        else: # print is another logger
            assert file is None, '`file` cannot be given if logger is a sublogger of another logger'
            
        if isinstance(file, str) or isinstance(file, Path):
            self.file = open(file, 'w')
        else: # can only assume file is either an actual file or None
            self.file = file
            
        self.prefix = prefix
        
        self.display = display
        
    def queue(self, *a, caller=None, **k):
        if caller is None:
            caller = self
        
        if isinstance(self.print, __class__):
            self.print.queue(*a, caller=caller, **k)
        else:
            self._queue.append((caller, a, k))
        
    def __call__(self, *a, display=None, file=None, **k):
        display = self.display if display is None else display
        file = self.file if file is None else file
        
        prepend = () if self.prefix is None else (self.prefix,)
        
        if file is None or file:
            if isinstance(self.print, __class__):
                k['display'] = False
            self.print(*(prepend+a), file=file, **k)
        
        if display is None or display:
            if isinstance(self.print, __class__):
                k['display'] = True
                k['file'] = False
            self.print(*(prepend+a), **k)
            
        self.dequeue()
        
    def dequeue(self):
        if hasattr(self, '_queue') and self._queue:
            queue = self._queue
            self._queue = [] # important to avoid infinite recursion when calling self
            for caller, a_, k_ in queue:
                caller(*a_, **k_)
                
    def logger(self, prefix=None, **k):
        return __class__(print=self, prefix=prefix, **k)
    
    def __del__(self):
        if self.file is not None:
            self.file.close()